In [ ]:
import os
os.chdir('/Users/huhu/Desktop/Data Science for Design/Assignment 3 Project + Report 1000 + Video 2mins 12.7')
os.listdir()
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np

Data Description
After reading in the data from PA-X Data, we can describe our data. These data are selected, since the challenge in this individual assignment is to visualise trends in violent events alongside peace agreement signings with the content of Date, Country, Natural Resources, and Environment. The links between these data and the real world can be seen in this way, if the value for NatRes or LaEn is 1, the peace agreement involves natural resources or environment factors. If the value for NatRes or LaEn is 0, natural resources or environment factors are not contained in the peace agreement.

In [ ]:
pec_df = pd.read_excel('peace agreement.xlsx', index_col='Dat', parse_dates=['Dat']) # read data


In [ ]:
pec_df = pec_df[['Con','NatRes', 'LaEn']]


In [ ]:
pec_df.dtypes # data types

In [ ]:
pec_df.shape # dimension

In [ ]:
pec_df.size # data size

In [1]:
pec_df.info() # data information

NameError: name 'pec_df' is not defined

In [ ]:
np.max(pec_df.index),np.min(pec_df.index) # time span

In [ ]:
pec_df.describe() # summary statistics

# Data Exploration
First, we select those records including NatRes/Enviroment. To initialize our analysis, we select top 10 Countries/Regions according to the Number of agreements signed.

In [1]:
"""
Select Top 10 Countries/Regions according to the Number of agreements signed
"""
countries = pec_df[((pec_df.NatRes==1) | (pec_df.LaEn == 1))].Con.values
countries = [i.split("/") for i in countries]

countries = list(itertools.chain.from_iterable(countries))
countries = [i.split("-") for i in countries]
countries = list(itertools.chain.from_iterable(countries))
countries = [i.replace('(', '').replace(')', '') for i in countries]

name, counts = np.unique(countries, return_counts=True)
cty_df = pd.DataFrame([name, counts], index=['countries', 'counts']).T
most_agreements_cty = cty_df.sort_values('counts', ascending= False).head(10)
most_agreements_cty.set_index('countries').plot(kind = 'bar',figsize = (16,8),rot =35)
plt.title('Top 10 Regions with Natural Resource/Enviroment Relevant Agreement')
plt.ylabel("Number")
plt.show()

NameError: name 'pec_df' is not defined